In [19]:
import matplotlib.pyplot as plt
from pydoc import doc
import re 
import os
from collections import Counter
import pandas as pd
from ast_gloss import ast_parse
from tqdm import tqdm
import numpy as np
import ast
import torch
from torch.utils.data import Dataset

In [39]:
manifests = os.listdir('files')
manifests = manifests.remove('cubert_metadata000000000000')
print ('list of bigquery files:', manifests)
#script_df = pd.read_csv('files/cubert_metadata000000000000')
#smaller test script
script_df = pd.read_csv('files/bq_sample.csv')
'''
for manifest in manifests:
    temp_df = pd.read_csv(f'files/{manifest}')
    script_df = pd.concat((script_df, temp_df))
'''


#display (script_df)

list of bigquery files: None


,id,repository,license,filepath,content,forks,issue_events,stars
0,246d6f096c1e6395a9b339160a5d871ccedfe943,swift-nav/libsbp,lgpl-3.0,python/sbp/client/__init__.py,# Copyright (C) 2015 Swift Navigation Inc.\n# ...,111,257,61
1,50e1fac147d3235cdc012adb487a0e7e6c1c71f2,tow/sunburnt,mit,sunburnt/test_strings.py,from .strings import RawString\n\n\ndef test_s...,47,29,114
2,bfa14468706050e1ac1206594dd0261da672358e,Akagi201/learning-python,mit,dnspython/e164.py,#!/usr/bin/env python\n\nimport dns.e164\n\nn ...,124,2,85
3,2d0f571c6956e9cbf41d75e5dd51cbda68f27e29,Sprytile/Sprytile,mit,rx/linq/observable/switchlatest.py,"from rx.core import Observable, AnonymousObser...",79,81,735
4,ecba837af47b5645206b1d070b26c854e745eeb1,lukasmartinelli/py14,mit,py14/analysis.py,"import ast\n\n\ndef add_imports(node):\n """"...",70,1,631
...,...,...,...,...,...,...,...,...
11025,d9357f9b75dffbbcb5e335334239a8413762152a,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/sipvicious/pptable.py,"import cStringIO,operator\n\ndef indent(rows, ...",156,2,626
11026,603473484f912750f7233183d58ed846e58c83e3,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/voiper/sulley/sulley/pedrpc.py,import sys\nimport struct\nimport socket\nimpo...,156,2,626
11027,cc99df861cbcff2852091f0e6e76a48d111d9d47,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/wifite/wifite.py,#!/usr/bin/python\n# -*- coding: cp1252 -*-\n\...,156,2,626
11028,0412701d3031478948082b3ac86b6e803acd3203,sfwa/ukf,mit,examples/ahrs/python/ukf/__init__.py,#Copyright (C) 2013 Daniel Dyer\n#\n#Permissio...,156,30,322


In [40]:
from ast_gloss import ast_parse
# get metrics
def count_casing(input_code):
    case_dict = {"lowerCamelCase": 0, "UpperCamelCase": 0, "snake_case": 0}
    lower_camel = r"[a-z]+([A-Z][a-z0-9]+)+"
    upper_camel = r"[A-Z][a-z]+([A-Z][a-z0-9]+)+"
    snake = r"[a-z]+(_[a-z0-9]+)+"
    input_code = input_code.replace("(", " ")
    input_code = input_code.replace(")", " ")
    for token in input_code.split():
        if re.match(lower_camel, token):
            case_dict["lowerCamelCase"] += 1
        elif re.match(upper_camel, token):
            case_dict["UpperCamelCase"] += 1
        elif re.match(snake, token):
            case_dict["snake_case"] += 1
    return case_dict


def count_docstrings(input_code):
    docstr = r"\"\"\"[\s\S]*?\"\"\"|\'\'\'[\s\S]*?\'\'\'"
    search = re.findall(docstr, input_code)
    doc_len = 0
    for docstring in search:
        doc_len += len(docstring)
    return len(search), doc_len


def count_comments(input_code):
    comment = r"#.*"
    search = re.findall(comment, input_code)
    comment_len = 0
    for comment in search:
        comment_len += len(comment)
    return len(search), comment_len

def file_probe(filename, script):
    '''
    Function takes a script in text form and calculates metrics for a given script. 
    This is based off of the github gloss and may need to have regexes made consistent with py150
        * # of docstrings TODO - Count docstrings per method / how many methods have docstrings
        * # of comments
        * Average docstring length (words)
        * Average comment length (words)
        * Average comment density
    '''
    comment_count = 0
    comment_length = 0
    docstring_count = 0
    docstring_length = 0
    line_count = 0
    comment_density =  0
    docstring_density =  0
    eval_dict = Counter()
    code = str(script)
    
    line_count += sum([1 for _ in ((code)).split("\n")])
    num_comments, comments_lengths = count_comments(code)
    eval_dict['total_num_files'] = 1
    try:
        parse = ast_parse(code)
        eval_dict += parse
        #print('success')
        
    except:
        #print(f"Parse error on file {filename}, skipping.")
        return { 'error' : "Parse error on file {filename}, skipping."}
    file_count = eval_dict["total_num_files"]
    var_count = eval_dict["var_total"]
    eval_dict['average_comment_count']  =  comment_count    / file_count  # average comment count over files
    if eval_dict['average_comment_count'] != 0:      
        eval_dict['average_comment_length'] =  comment_length   / comment_count # average length of comments
    else:
        eval_dict['average_comment_length'] = 0
    eval_dict['comment_line_density']   =  comment_count / line_count # average number of commented lines

    total_methods = eval_dict[ast.FunctionDef] + eval_dict[ast.ClassDef]
    if total_methods: # normalizes docstrings over number of classes AND functions 
      eval_dict['methods_with_docstrings'] = eval_dict["ds_count"] / total_methods

    if var_count: # gets casing ratio over all variable counts
      eval_dict['ratio_snake_case'] = eval_dict['snake_case'] / var_count
      eval_dict['ratio_locamel_case'] = eval_dict['lower_camel'] / var_count
      eval_dict['ratio_upcamel_case'] = eval_dict['upper_camel'] / var_count
      eval_dict['ratio_lower_case'] = eval_dict['lower'] / var_count

    eval_dict['lines_per_file']   =  line_count / eval_dict["total_num_files"]
    
    #added metrics for 4/22
    eval_dict['async_functions_per_file'] = eval_dict[ast.AsyncFunctionDef]
    eval_dict['functions_per_file'] = eval_dict[ast.FunctionDef]
    eval_dict['classes_per_file'] =eval_dict[ast.ClassDef]
    if eval_dict['classes_per_file'] != 0:
      eval_dict['parents_per_class'] = eval_dict['num_parents'] / eval_dict['classes_per_file']
    if eval_dict['functions_per_file'] != 0:
      eval_dict['decorators_per_function'] = eval_dict['num_decorators_func'] / eval_dict['functions_per_file']
    if eval_dict['classes_per_file'] != 0:
      eval_dict['decorators_per_class'] = eval_dict['num_decorators_class'] / eval_dict['classes_per_file']
    #cleanup things with ast names for clarity sake
    del eval_dict[ast.FunctionDef]
    del eval_dict[ast.ClassDef]
    del eval_dict[ast.Name]
    del eval_dict['num_parents']
    del eval_dict['num_decorators_class'] 
    del eval_dict['num_decorators_func'] 
    del eval_dict[ast.AsyncFunctionDef]
    
    return eval_dict

In [41]:
#run metrics
script_count = len(script_df['filepath'])
filenames = script_df['filepath']
scripts = script_df['content']
internal_metrics = []
parse_error = []
for i in range (0, script_count):
    filename = filenames[i]
    script = scripts[i]
    metrics = file_probe(filename, script)
    if 'error' in metrics.keys():
        parse_error.append(0)
    else:
        parse_error.append(1)
    internal_metrics.append(metrics)

In [42]:
# add internal metrics to df and filter out parse errors
script_df['internal_metrics'] = internal_metrics
script_df['error'] = parse_error
#
script_df = script_df[script_df['error'] == 1]
display(script_df)

,id,repository,license,filepath,content,forks,issue_events,stars,internal_metrics,error
0,246d6f096c1e6395a9b339160a5d871ccedfe943,swift-nav/libsbp,lgpl-3.0,python/sbp/client/__init__.py,# Copyright (C) 2015 Swift Navigation Inc.\n# ...,111,257,61,"{'total_num_files': 1, 'average_comment_count'...",1
1,50e1fac147d3235cdc012adb487a0e7e6c1c71f2,tow/sunburnt,mit,sunburnt/test_strings.py,from .strings import RawString\n\n\ndef test_s...,47,29,114,"{'total_num_files': 1, 'ds_count': 1, 'ds_char...",1
3,2d0f571c6956e9cbf41d75e5dd51cbda68f27e29,Sprytile/Sprytile,mit,rx/linq/observable/switchlatest.py,"from rx.core import Observable, AnonymousObser...",79,81,735,"{'total_num_files': 1, 'ds_count': 1, 'ds_char...",1
4,ecba837af47b5645206b1d070b26c854e745eeb1,lukasmartinelli/py14,mit,py14/analysis.py,"import ast\n\n\ndef add_imports(node):\n """"...",70,1,631,"{'total_num_files': 1, 'ds_count': 5, 'ds_char...",1
5,086d0cae3ebf7c8e0d9f26ddf3cc313c8645e52b,vintasoftware/django-role-permissions,mit,docs/conf.py,# -*- coding: utf-8 -*-\n#\n# django-role-perm...,122,129,561,"{'total_num_files': 1, 'var_total': 17, 'lower...",1
...,...,...,...,...,...,...,...,...,...,...
11021,9aa4e1f5a87899c9241b8eadddbe87d260f7efb4,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/metagoofil/hachoir_parser/version.py,"__version__ = ""1.2.1""\nPACKAGE = ""hachoir-pars...",156,2,626,"{'total_num_files': 1, 'var_total': 4, 'averag...",1
11022,b9e06ded665a9cb4ca217eca01d50889cc037681,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/sqlmap/plugins/dbms/sqlite/takeove...,"#!/usr/bin/env python\n\n""""""\nCopyright (c) 20...",156,2,626,"{'total_num_files': 1, 'var_total': 17, 'upper...",1
11023,0020ec2012b4b27a2334017af39796238ffc66f1,pwnieexpress/raspberry_pwn,gpl-3.0,src/pentest/goodfet/GoodFETSPI.py,#!/usr/bin/env python\n# GoodFET SPI and SPIFl...,156,2,626,"{'total_num_files': 1, 'var_total': 95, 'ds_co...",1
11028,0412701d3031478948082b3ac86b6e803acd3203,sfwa/ukf,mit,examples/ahrs/python/ukf/__init__.py,#Copyright (C) 2013 Daniel Dyer\n#\n#Permissio...,156,30,322,"{'total_num_files': 1, 'var_total': 251, 'lowe...",1


In [43]:
#internal metrics are generated now and everything is stored in dataframe should be ready for clustering

In [44]:
#script is in df
#df to data set
class BigQueryDataset(Dataset):
    def __init__(self, script_df, clustering_result):
        self.script_df = script_df
        self.clustering_result = clustering_result
#can add other things here as well

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.script_df['count'][idx]
        label = self.clustering_result[idx] #this is where clustering result would go

        return image, label